In [2]:
!pip install dash

     |████████████████████████████████| 8.5 MB 12.4 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 41.0 MB/s eta 0:00:01
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=425853a59aac97d3d6a45b2a221eccd82aba9d9b7d53d9b1e7624a3472609b21
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/2b/37/e9/4b5a8c878a47590de4dcf6bdc5a158788a1c17bea578a3ab60
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=6295a1e810a33dfc9584d7299da6d40320b3fbd06ec1ed752e4cf8f234341c21
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/4d/22/3e/d58dc9820aeffc7b7193d04a63b08f585eef4a1ce836d69141
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=01301e120ba9b9ca21186bd1a0a91f81a9155b1f8f09ed4e6bb42fc946a53138
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/1f/9

In [3]:
!pip install wget

In [5]:
import wget
wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')

'spacex_launch_dash.csv'

In [6]:
!pip install jupyter_dash

  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11448 sha256=603aa6890d8af52524ffd9754d7c54cc9b0520da3800253753d0bd39268dd5ca
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/ce/18/7f/e9527e3e66db1456194ac7f61eb3211068c409edceecff2d31
Successfully built retrying


In [7]:
!pip install typing-extensions

In [ ]:
# Import required libraries
import pandas as pd
from jupyter_dash import JupyterDash
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                             options=[{'label': 'All Sites', 'value': 'ALL'},
                                                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                      {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                      {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                      {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                             value='ALL',
                                             placeholder='Select a Launch Site here',
                                             searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0:'0',
                                                       2500:'2500',
                                                       5000:'5000',
                                                       7500:'7500',
                                                       10000:'10000'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site].groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        fig=px.pie(filtered_df, values='class count',
        names='class', 
        title='Total Success Launches for site {}'.format(entered_site))
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(entered_site,entered_payloads):
    sel_payload_min = entered_payloads[0]
    sel_payload_max = entered_payloads[1]
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= sel_payload_min) & (spacex_df['Payload Mass (kg)'] <= sel_payload_max)]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)',y='class',
        color='Booster Version Category', 
        title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        filtered_dfs = filtered_df[filtered_df['Launch Site']==entered_site]
        fig=px.scatter(filtered_dfs, x='Payload Mass (kg)',y='class', 
        color='Booster Version Category',
        title=f'Correlation between Payload and Success for site {entered_site}')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)


/tmp/wsuser/ipykernel_609/830389252.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/tmp/wsuser/ipykernel_609/830389252.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
